In [17]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
import glob

# Chemins vers les dossiers pop et techno
pop_dir = "./Data/genres_original/classical/"
techno_dir = "./Data/genres_original/metal/"

# Taille fixe pour les spectrogrammes (à ajuster selon vos besoins)
input_shape = (128, 128)

# Fonction pour charger les spectrogrammes à partir des fichiers audio
def load_spectrogram(file_path, input_shape):
    y, sr = librosa.load(file_path, sr=None)  # Charge le fichier audio
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)  # Convertit en spectrogramme
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)  # Normalise les valeurs
    # Redimensionne le spectrogramme pour correspondre à la taille fixe
    spectrogram = spectrogram[:input_shape[0], :input_shape[1]]
    # Redimensionne pour avoir les dimensions attendues par le modèle
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram

# Fonction pour charger toutes les données d'un répertoire
def load_data_from_directory(directory, label, input_shape):
    data = []
    for filename in glob.glob(os.path.join(directory, '*.wav')):
        spectrogram = load_spectrogram(filename, input_shape)
        data.append((spectrogram, label))
    return data

# Chargement des données pour le genre pop
pop_data = load_data_from_directory(pop_dir, label=0, input_shape=input_shape)
# Chargement des données pour le genre techno
techno_data = load_data_from_directory(techno_dir, label=1, input_shape=input_shape)

# Fusion des données
all_data = pop_data + techno_data

# Séparation des données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)

# Séparation des données d'entraînement en données et étiquettes
train_data, train_labels = zip(*train_data)
train_data = np.array(train_data)
train_labels = np.array(train_labels)

# Séparation des données de test en données et étiquettes
test_data, test_labels = zip(*test_data)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

print("Données d'entraînement:", train_data.shape)
print("Étiquettes d'entraînement:", train_labels.shape)
print("Données de test:", test_data.shape)
print("Étiquettes de test:", test_labels.shape)


Données d'entraînement: (160, 128, 128, 1)
Étiquettes d'entraînement: (160,)
Données de test: (40, 128, 128, 1)
Étiquettes de test: (40,)


In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Définition des dimensions des spectrogrammes
input_shape = train_data.shape[1:]  # Shape de chaque spectrogramme (sans le nombre d'échantillons)

# Création du modèle CNN
def create_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Couche de sortie avec activation sigmoid pour la classification binaire
    ])
    return model

# Création du modèle
model = create_model(input_shape)

# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary Crossentropy pour la classification binaire
              metrics=['accuracy'])

# Entraînement du modèle
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_split=0.2)

# Évaluation du modèle
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)


Epoch 1/10


4/4 [==============================] - 6s 472ms/step - loss: 11.8678 - accuracy: 0.4688 - val_loss: 5.6384 - val_accuracy: 0.4688
Epoch 2/10
4/4 [==============================] - 1s 257ms/step - loss: 2.9533 - accuracy: 0.4375 - val_loss: 0.7077 - val_accuracy: 0.4688
Epoch 3/10
4/4 [==============================] - 1s 258ms/step - loss: 0.7122 - accuracy: 0.5469 - val_loss: 0.6701 - val_accuracy: 0.4688
Epoch 4/10
4/4 [==============================] - 1s 258ms/step - loss: 0.9241 - accuracy: 0.4531 - val_loss: 0.6857 - val_accuracy: 0.5312
Epoch 5/10
4/4 [==============================] - 1s 254ms/step - loss: 0.6771 - accuracy: 0.5312 - val_loss: 0.6002 - val_accuracy: 0.5000
Epoch 6/10
4/4 [==============================] - 1s 249ms/step - loss: 0.5447 - accuracy: 0.8594 - val_loss: 0.5910 - val_accuracy: 0.4688
Epoch 7/10
4/4 [==============================] - 1s 251ms/step - loss: 0.5336 - accuracy: 0.7422 - val_loss: 0.4915 - val_accuracy: 0.7812
Epoch 8/10
4/4 [=

In [19]:
import os
import librosa
import glob

classical_music_dir = "./Data/genres_original/classical/"
metal_music_dir = "./Data/genres_original/metal/"

# Charger les données musicales classiques
classical_music_files = glob.glob(classical_music_dir +'*.wav')

# Charger les données musicales metal
metal_music_files = glob.glob(metal_music_dir+ '*.wav')


In [25]:
!pip install magenta

Defaulting to user installation because normal site-packages is not writeable
  Using cached magenta-2.1.4-py3-none-any.whl (1.4 MB)
  Using cached dm_sonnet-2.0.0-py3-none-any.whl (254 kB)
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached tensorflow-2.9.1-cp39-cp39-win_amd64.whl (444.0 MB)
  Using cached matplotlib-3.5.2-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached pretty_midi-0.2.9-py3-none-any.whl
  Using cached imageio-2.20.0-py3-none-any.whl (3.4 MB)
  Using cached python-rtmidi-1.1.2.tar.gz (204 kB)
  Using cached note_seq-0.0.3-py3-none-any.whl (210 kB)
  Using cached scikit_image-0.19.3-cp39-cp39-win_amd64.whl (12.1 MB)
  Using cached numba-0.49.1.tar.gz (2.0 MB)
  Using cached tensorflow_datasets-4.6.0-py3-none-any.whl (4.3 MB)
  Using cached sox-1.4.1-py2.py3-none-any.whl (39 kB)
  Using cached mir_eval-0.7-py3-none-any.whl
  Using cached pygtrie-2.5.0-py3-none-any.whl (25 kB)
  Using cached sk_video-1.1.10-py2.py3-none-any.whl (2.3 MB)
  Using cach

  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Miniconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Maure\\AppData\\Local\\Temp\\pip-install-31eiy25y\\numba_57041d81eabf42f3990df2e72785a6ba\\setup.py'"'"'; __file__='"'"'C:\\Users\\Maure\\AppData\\Local\\Temp\\pip-install-31eiy25y\\numba_57041d81eabf42f3990df2e72785a6ba\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Maure\AppData\Local\Temp\pip-wheel-iwhdckoa'
       cwd: C:\Users\Maure\AppData\Local\Temp\pip-install-31eiy25y\numba_57041d81eabf42f3990df2e72785a6ba\
  Complete output (826 lines):
  TBB not found
  OpenMP disabled
  running bdist_wheel
  running build
  got version from file C:\Users\Maure\AppData

In [26]:
import magenta.models.music_vae as music_vae

music_vae_model = music_vae.MusicVAE("cat-mel_2bar_big")


ModuleNotFoundError: No module named 'magenta'

In [ ]:
def generate_metal_music(classical_music_input):
    # Charger la musique classique en utilisant le modèle de Magenta
    sequence = music_vae_model.improvise(classical_music_input)
    # Convertir la séquence MIDI en audio
    audio = sequence.to_note_sequence().to_audio()
    return audio

# Exemple d'utilisation
classical_music_input = librosa.load(classical_music_files[0], sr=None)
metal_music_output = generate_metal_music(classical_music_input)


In [ ]:
import soundfile as sf

# Spécifiez le chemin de destination où vous souhaitez enregistrer le fichier audio généré
destination_path = "./metal_music_generated.wav"

# Enregistrez le fichier audio généré
sf.write(destination_path, metal_music_output, sample_rate)


In [31]:

# Chemins vers les dossiers pop et techno
pop_dir = "./Data/genres_original/classical/"
techno_dir = "./Data/genres_original/metal/"

# Taille fixe pour les spectrogrammes (à ajuster selon vos besoins)
input_shape = (128, 128)

# Fonction pour charger les spectrogrammes à partir des fichiers audio
def load_spectrogram(file_path, input_shape):
    y, sr = librosa.load(file_path, sr=None)  # Charge le fichier audio
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)  # Convertit en spectrogramme
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)  # Normalise les valeurs
    # Redimensionne le spectrogramme pour correspondre à la taille fixe
    spectrogram = spectrogram[:input_shape[0], :input_shape[1]]
    # Redimensionne pour avoir les dimensions attendues par le modèle
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    return spectrogram

# Fonction pour charger toutes les données d'un répertoire
def load_data_from_directory(directory, label, input_shape):
    data = []
    i =0
    for filename in glob.glob(os.path.join(directory, '*.wav')):
        if i == 0 :
            spectrogram = load_spectrogram(filename, input_shape)
            data.append((spectrogram, label))
            return spectrogram

# Chargement des données pour le genre pop
pop_data = load_data_from_directory(pop_dir, label=0, input_shape=input_shape)
# Chargement des données pour le genre techno
techno_data = load_data_from_directory(techno_dir, label=1, input_shape=input_shape)

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Génération de données d'exemple (remplacez cette étape par votre propre processus de chargement de données)
# Exemple de spectrogramme de musique classique
classical_spectrogram = pop_data  # Exemple de spectrogramme classique (128x128)
# Exemple de spectrogramme de musique metal (généré à partir du spectrogramme classique)
metal_spectrogram = techno_data  # Exemple de spectrogramme metal (128x128)
print(classical_spectrogram)
# Définition du modèle
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),  # Couche de sortie (128 unités pour les valeurs du spectrogramme)
])

# Compilation du modèle
model.compile(optimizer='adam', loss='mse')  # Utilisation de la perte MSE pour la reconstruction du spectrogramme

# Entraînement du modèle
model.fit(classical_spectrogram, metal_spectrogram, epochs=10, batch_size=32)

# Génération d'un spectrogramme metal à partir d'un spectrogramme classique
classical_input_spectrogram = np.random.rand(1, 128, 128, 1)  # Exemple de spectrogramme classique d'entrée
predicted_metal_spectrogram = model.predict(classical_input_spectrogram)

# Affichage des dimensions du spectrogramme prédit
print("Dimensions du spectrogramme metal prédit:", predicted_metal_spectrogram.shape)



import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt


# Inversion du spectrogramme pour obtenir le signal audio
reconstructed_audio = librosa.griffinlim(predicted_metal_spectrogram)

# Exemple de chargement des données audio d'origine pour la fréquence d'échantillonnage
# (remplacez cette étape par votre propre processus de chargement)
# original_audio, sr = librosa.load("chemin/vers/votre/fichier/audio.wav", sr=None)

# Écriture du fichier WAV
librosa.output.write_wav("reconstructed_audio.wav", reconstructed_audio, sr)


[[[-35.17282 ]
  [-33.15856 ]
  [-33.095356]
  ...
  [-35.711678]
  [-35.781322]
  [-39.535355]]

 [[-28.473568]
  [-25.597416]
  [-23.591324]
  ...
  [-28.219748]
  [-27.288116]
  [-27.683477]]

 [[-27.32995 ]
  [-22.706562]
  [-21.004128]
  ...
  [-21.248734]
  [-21.515327]
  [-23.237494]]

 ...

 [[-80.      ]
  [-80.      ]
  [-80.      ]
  ...
  [-80.      ]
  [-80.      ]
  [-80.      ]]

 [[-80.      ]
  [-80.      ]
  [-80.      ]
  ...
  [-80.      ]
  [-80.      ]
  [-80.      ]]

 [[-80.      ]
  [-80.      ]
  [-80.      ]
  ...
  [-80.      ]
  [-80.      ]
  [-80.      ]]]
Epoch 1/10


ValueError: in user code:

    File "C:\Users\Maure\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Maure\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Maure\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Maure\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Maure\AppData\Roaming\Python\Python39\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'conv